### 分组
#### 分组模式及其对象
分组必须要明确三个要素，分组依据、数据来源、操作及其返回结果，使用的函数是groupby(分组依据）[数据来源].使用操作


In [1]:
import pandas as pd
df=pd.read_csv("train_k.csv")

In [2]:
df.head()

,PhoneService,StreamingTV,StreamingMovies,PaperlessBilling,MonthlyCharges,MultipleLines_Yes,InternetService_Fiber optic,PaymentMethod_Electronic check,tenure_2,Churn
0,0,0,0,1,29.85,0,0,1,0,0
1,1,0,0,0,56.95,0,0,0,0,0
2,1,0,0,1,53.85,0,0,0,0,1
3,0,0,0,0,42.30,0,0,0,0,0
4,1,0,0,1,70.70,0,1,1,0,1


In [3]:
df.groupby("Churn")["MonthlyCharges"].mean()

Churn
0    61.344878
1    74.604308
Name: MonthlyCharges, dtype: float64

### 分组依据的本质

In [4]:
df.groupby(["Churn","PaperlessBilling"])["MonthlyCharges"].mean()

Churn  PaperlessBilling
0      0                   49.925898
       1                   71.213845
1      0                   63.378755
       1                   78.364989
Name: MonthlyCharges, dtype: float64

### 在分组的时候也可以对数据分组的依据进行设置条件，这里需要根据数据集的真实情况来实施

In [8]:
df[["Churn","PaperlessBilling"]].drop_duplicates()
# 这里是删除某些重复的行

,Churn,PaperlessBilling
0,0,1
1,0,0
2,1,1
18,1,0


### groupby对象
- ngroups 属性可以得到分组的个数
- group属性，可以返回从组名映射到组索引列表的字典
- size这个属性在分组中的应用表示统计每个分组的元素个数

 ### 分组的三大操作  --聚合、变换和过滤
 ## 聚合函数

### 内置聚合函数
- agg函数

In [9]:
gd=df.groupby("Churn")["MonthlyCharges"]

In [11]:
gd.idxmin()

Churn
0    3711
1    6474
Name: MonthlyCharges, dtype: int64

In [15]:
gd.quantile(0.95)

Churn
0    108.4425
1    105.6500
Name: MonthlyCharges, dtype: float64

In [17]:
gd.agg(['sum', 'idxmax', 'skew'])

,sum,idxmax,skew
Churn,,,
0,316784.95,4575,-0.028313
1,138540.20,5116,-0.735405


### 对特定的列使用聚合函数
#### 可以对聚合的结果进行重命名

In [21]:
gd.agg(lambda x: x.mean()-x.min())

Churn
0    43.094878
1    55.754308
Name: MonthlyCharges, dtype: float64

### 变换和过滤
- transform方法与变换函数

In [22]:
gd.cummax().head()
# 每组累计最大值

0    29.85
1    56.95
2    53.85
3    56.95
4    70.70
Name: MonthlyCharges, dtype: float64

In [25]:
gd.transform(lambda x :(x-x.mean())/x.std()).head()
# x值减去平均值再除以标准差


0   -1.013681
1   -0.141452
2   -0.843757
3   -0.612970
4   -0.158728
Name: MonthlyCharges, dtype: float64

In [26]:
gd.transform("mean").head()

0    61.344878
1    61.344878
2    74.604308
3    61.344878
4    74.604308
Name: MonthlyCharges, dtype: float64

### 组索引与过滤
#### 定义了 filter 方法进行组的筛选，其中自定义函数的输入参数为数据源构成的 DataFrame 本身，在之前例子中定义的 groupby 对象中，传入的就是 df[['Height', 'Weight']] ，因此所有表方法和属性都可以在自定义函数中相应地使用，同时只需保证自定义函数的返回为布尔值即可

In [27]:
gd.filter(lambda x :x.shape[0]>100).head()

0    29.85
1    56.95
2    53.85
3    42.30
4    70.70
Name: MonthlyCharges, dtype: float64

### 跨列分组
- apply的引入


In [29]:
gd.head()

0      29.85
1      56.95
2      53.85
3      42.30
4      70.70
5      99.65
6      89.10
7      29.75
8     104.80
13    103.70
Name: MonthlyCharges, dtype: float64

In [30]:
df.head()

,PhoneService,StreamingTV,StreamingMovies,PaperlessBilling,MonthlyCharges,MultipleLines_Yes,InternetService_Fiber optic,PaymentMethod_Electronic check,tenure_2,Churn
0,0,0,0,1,29.85,0,0,1,0,0
1,1,0,0,0,56.95,0,0,0,0,0
2,1,0,0,1,53.85,0,0,0,0,1
3,0,0,0,0,42.30,0,0,0,0,0
4,1,0,0,1,70.70,0,1,1,0,1


In [37]:
def BMI(x):
    monthlycharges = x['MonthlyCharges']/30
    churn = x['Churn']
    BMI_value = monthlycharges/churn**2
    return BMI_value.mean()

In [39]:
# df.apply(BMI)